In [ ]:
# !pip install tensorflow

## Importing all Necessary Libraries

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

## Reading the DataSet

In [ ]:
df = pd.read_csv(r'C:\Users\aravindv\Documents\Github-projects\ML_PredictionsinHTML\House_Price_Pred\data.csv', na_values=['NA','?'])

df.head()

### Checking for any null values and Removing the unwanted Columns from the Dataset

In [ ]:
df.isnull().sum()
# IF you find any null values then you can use the below code to set the null values with median value of that particular column. 
#This is how you fill the null values.
# you can use medianm mean any thing that you want.

# df['floors'] = df['floors'].fillna(df['floors'].median())

In [ ]:
df.drop(df.columns[[0, 14,16,17]], axis=1, inplace=True)
df.head()

In [ ]:
df .info() # To find if there is any column that is not int or float.

## Converting categorical variables of the dataset into numerical variables - using ONE HOT ENCODING technique

In [ ]:
df.head()

In [ ]:
# Helps fetching the unique value for the column city.
df.city.unique()

In [ ]:
df['city']=df['city'].apply({'Shoreline':0,'Seattle':1,'Kent':2,'Bellevue':3,'Redmond':4,'Maple Valley':5,'North Bend':6,'Lake Forest Park':7,
                                 'Sammamish':8,'Auburn':9,'Des Moines':10,'Bothell':11,'Federal Way':12,'Kirkland':13,'Issaquah':14,
                                 'Woodinville':15,'Normandy Park':16,'Fall City':17,'Renton':18,'Carnation':19,'Snoqualmie':20,
                                 'Duvall':21,'Burien':22,'Covington':23,'Inglewood-Finn Hill':24,'Kenmore':25,'Newcastle':26,'Mercer Island':27,
                                 'Black Diamond':28,'Ravensdale':29,'Clyde Hill':30,'Algona':31,'Skykomish':32,'Tukwila':33,'Vashon':34,
                                 'Yarrow Point':35,'SeaTac':36,'Medina':37,'Enumclaw':38,'Snoqualmie Pass':39,'Pacific':40,'Beaux Arts Village':41,
                                'Preston':42,'Milton':43}.get)

In [ ]:
df.head()

## Dividing the dataset into dependent and independent columns

In [ ]:
X= df.drop('price',axis=1)
y=df['price']

In [ ]:
X

In [ ]:
y

## Splitting the dataset into training and testing set

### 20% of the dataset will be used for testing(evaluation) and 80% of the data will be used for training purposes and setting random_state so the same test dataset will always be used for predicting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# This tells the nmber of Data and columns in the dataset "13 represents the column"
X_train.shape

In [ ]:
model = Sequential()
model.add(Dense(25, input_dim=X.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)

model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)

## Prediction and getting the score

In [ ]:
prediction = model.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(prediction, y_test))
print(f'After the training the score is: {score}')

## To save the Model

In [ ]:
os.chdir(r'C:\Users\aravindv\Documents\Github-projects\ML_PredictionsinHTML\House_Price_Pred')

In [ ]:
os.getcwd()

In [ ]:
model.save(os.path.join(os.getcwd(),"housepred_model.h5"))

## To find the Minimum and Maximum value of the dataset

In [ ]:
# For Multiple columns to exclude use,
# cols = [x for x in df.columns if x not in ('mpg', 'name')]
# cols

cols = [X for X in df.columns if X not in ('price')]
cols

print("{")
for i,name in enumerate(cols):
    print(f'"{name}":{{"min":{df[name].min()},"max":{df[name].max()}}}{"," if i<(len(cols)-1) else ""}')
print("}")

In [ ]:
! pip install flask

In [7]:
from tensorflow.keras.models import load_model
from flask import Flask,request, jsonify
import os
import numpy as np
import uuid

app = Flask(__name__)

In [8]:
Expected = {
"bedrooms":{"min":0.0,"max":9.0},
"bathrooms":{"min":0.0,"max":8.0},
"sqft_living":{"min":370,"max":13540},
"sqft_lot":{"min":638,"max":1074218},
"floors":{"min":1.0,"max":3.5},
"waterfront":{"min":0,"max":1},
"view":{"min":0,"max":4},
"condition":{"min":1,"max":5},
"sqft_above":{"min":370,"max":9410},
"sqft_basement":{"min":0,"max":4820},
"yr_built":{"min":1900,"max":2014},
"yr_renovated":{"min":0,"max":2014},
"city":{"min":0,"max":43}
}

## Load the saved Model

In [9]:
os.chdir(r'C:\Users\aravindv\Documents\Github-projects\ML_PredictionsinHTML\House_Price_Pred')

model = load_model(os.path.join(os.getcwd(),"housepred_model.h5"))

In [10]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 25)                350       
                                                                 
 dense_4 (Dense)             (None, 10)                260       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 621
Trainable params: 621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
@app.route('/api', methods=['POST'])
def houseprice_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f"Out of bounds: {name}, has value of: {value}, but it should be between {expected_min} and {expected_max}.")
        else:
            errors.append(f"Unexpected field: {name}.")
    for name in Expected:
        if name not in content:
            errors.append(f"Missing_value:{name}")
    if len(errors)<1:
        X= np.zeros((1,13))
        
        X[0,0] = content['price']
        X[0,1] = content['bedrooms']
        X[0,2] = content['bathrooms']
        X[0,3] = content['sqft_living']
        X[0,4] = content['sqft_lot']
        X[0,5] = content['floors']
        X[0,6] = content['waterfront']
        X[0,7] = content['view']
        X[0,8] = content['condition']
        X[0,9] = content['sqft_above']
        X[0,10] = content['sqft_basement']
        X[0,11] = content['yr_built']
        X[0,12] = content['yr_renovated']
        X[0,13] = content['city']
        
        prediction = model.predict(X)
        price = float(prediction[0])
        response = {"id": str(uuid.uuid4()), "price": price,"errors":errors}
    else:
        response = {"id": str(uuid.uuid4()), "errrors": errors}
        
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Sep/2022 14:54:29] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [04/Sep/2022 14:54:29] "GET /favicon.ico HTTP/1.1" 404 -


## Prediction Data

In [1]:
import os
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

os.chdir(r'C:\Users\aravindv\Documents\Github-projects\ML_PredictionsinHTML\House_Price_Pred')

model = load_model(os.path.join(os.getcwd(),"housepred_model.h5"))

In [6]:
df = pd.read_csv(r"C:\Users\aravindv\Documents\Github-projects\ML_PredictionsinHTML\House_Price_Pred\test.csv")
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city
0,2.0,3.0,1350,13500,2.0,0,1,2,1350,350,1925,2010,Federal Way


In [11]:
df['city']=df['city'].apply({'Shoreline':0,'Seattle':1,'Kent':2,'Bellevue':3,'Redmond':4,'Maple Valley':5,'North Bend':6,'Lake Forest Park':7,
                                 'Sammamish':8,'Auburn':9,'Des Moines':10,'Bothell':11,'Federal Way':12,'Kirkland':13,'Issaquah':14,
                                 'Woodinville':15,'Normandy Park':16,'Fall City':17,'Renton':18,'Carnation':19,'Snoqualmie':20,
                                 'Duvall':21,'Burien':22,'Covington':23,'Inglewood-Finn Hill':24,'Kenmore':25,'Newcastle':26,'Mercer Island':27,
                                 'Black Diamond':28,'Ravensdale':29,'Clyde Hill':30,'Algona':31,'Skykomish':32,'Tukwila':33,'Vashon':34,
                                 'Yarrow Point':35,'SeaTac':36,'Medina':37,'Enumclaw':38,'Snoqualmie Pass':39,'Pacific':40,'Beaux Arts Village':41,
                                'Preston':42,'Milton':43}.get)

In [12]:
df

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city
0,2.0,3.0,1350,13500,2.0,0,1,2,1350,350,1925,2010,12


In [13]:
arr = df.to_numpy()
arr

array([[2.000e+00, 3.000e+00, 1.350e+03, 1.350e+04, 2.000e+00, 0.000e+00,
        1.000e+00, 2.000e+00, 1.350e+03, 3.500e+02, 1.925e+03, 2.010e+03,
        1.200e+01]])

In [2]:
X = np.zeros((1,13))
X

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [3]:
X[0,0] = 3.0
X[0,1] = 2.00
X[0,2] = 1570
X[0,3] = 7500
X[0,4] = 2.0
X[0,5] = 0
X[0,6] = 4
X[0,7] = 5
X[0,8] = 3560
X[0,9] = 300
X[0,10] = 1932
X[0,11] = 2007
X[0,12] = 2

X

array([[3.000e+00, 2.000e+00, 1.570e+03, 7.500e+03, 2.000e+00, 0.000e+00,
        4.000e+00, 5.000e+00, 3.560e+03, 3.000e+02, 1.932e+03, 2.007e+03,
        2.000e+00]])

In [14]:
prediction = model.predict(arr)
float(prediction[0])

1/1 [==============================] - 0s 165ms/step


432862.6875

In [6]:
import requests
json = {"bedrooms":3.0,
       "bathrooms":2.00,
       "sqft_living": 1570,
       "sqft_lot": 7500,
       "floors": 2.0,
       "waterfront": 0,
       "view":4,
       "condition":5,
        "sqft_above":3560,
       "sqft_basement":300,
       "yr_built": 1932,
       "yr_renovated":2007,
       "city":2}

r  = requests.post("http://localhost:5000/api", json=json)
if r.status_code == 200:
    print("Works: {}".format(r.text))
else: 
    print("Fail:{}".format(r.text))

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000227D52C9640>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))